In [1]:
import requests 
import yaml
import json
import datetime as dt
import time
from azure.storage.blob import BlobServiceClient, BlobType

with open("api-key.yaml", "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = {
    "base_url":f"https://api.surveymonkey.com/v3/surveys/{data['sm']['survey-id']}", 
    "headers":{
        "Authorization": f"Bearer {data['sm']['access-token']}"
    }, 
    "survey-details-fp": "sm-survey-details.json" #  
}


# CareerOneStop Skills Matcher 
COS_DATA = {
    "url":f"https://api.careeronestop.org/v1/skillsmatcher/{data['cs']['user-id']}",
    "headers":{
        "Authorization": f"Bearer {data['cs']['token-key']}"
    }, 
    "survey-details-fp": "cos-survey-details.json"
}

## -- Cloud storage and logging -- ## 
# This is set currently to Azure but could change 
# We would have more sophisticated logging in production, 
# e.g. use a dedicated logging service or not just uploading to a bucket, 
# use multiple logfiles in a naming system, different logfiles for type of log data,
# logging alerts for certain kinds of log events and messages, etc.

# AZ_CONNECTION_STR = data['az']['connection-str']
# AZ_CONTAINER_NAME = data['az']['container-name']

# blob_service_client = BlobServiceClient.from_connection_string(AZ_CONNECTION_STR)
# container_client = blob_service_client.get_container_client(AZ_CONTAINER_NAME)
log_file = "logfile.txt" 

## Any files in the script which are currently being read locally (from within app environment) might be read from cloud storage instead.
## A cloud copy of each file should be maintained, at least. 

/Users/alexsocarras/tech_impact/career-onestop/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


---

**Set Up and Test Webhook**

In [2]:
url = "https://api.surveymonkey.com/v3/webhooks"

# Define the headers
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {data['sm']['access-token']}",
    "Content-Type": "application/json",
}

# Define the data payload
data = {
    "name": "Response Complete Webhook",
    "subscription_url": "https://surveymonkey.com/webhook_receiver",
    "authorization": "xyz", 
    "verify_ssl": False, # not safe for production
    "event_type": "response_completed",
    "object_type": "survey",
    "object_ids": [str(data['sm']['survey-id'])]
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    64  100    64    0     0    103      0 --:--:-- --:--:-- --:--:--   104


{"data": [], "per_page": 50, "page": 1, "total": 0, "links": {}}

---

In [16]:
## -- Util Functions -- ## 

## Logging wrapper 
def log_azure(log_data, log_file=log_file):
    return None # dummy function placeholder for testing until logging is configured 
    ## TO-DO: Logger should automatically add timestamp to log_data by default.
    """Log to Azure blob -- appends to logfile if it exists already."""
    # Check if log_file exists in container
    blob_client = container_client.get_blob_client(log_file)
    if not blob_client.exists():
        blob_client.upload_blob(log_data, blob_type=BlobType.AppendBlob)
    else:
        # Append the log data to the existing blob
        blob_properties = blob_client.get_blob_properties()
        offset = blob_properties.size
        blob_client.upload_blob(log_data, blob_type=BlobType.AppendBlob, length=len(log_data), offset=offset)

## GET request wrapper  
def get_request(url:str, headers:dict, params:dict):
    """Wrapper for get request with logging."""
    try:
        start_time = time.time()
        response = requests.get(url, headers, params)
        end_time = time.time()
        
        log_data = {
            "url":{url}, 
            "date":{dt.datetime.now()}, 
            "response_code":{response.status_code}, 
            "time_taken":f"{end_time - start_time:.2f}"
        }
        log_azure(log_data=log_data)

        return response

    except Exception as e:
        error_data = {
            "url": url,
            "date": dt.datetime.now().isoformat(),
            "error_message": str(e)
        }
        log_azure(log_data=error_data)

        raise e 

In [ ]:
## POST a webhook for when the survey is completed 

requests.get("https://api.surveymonkey.com/v3/webhooks", headers=headers)

In [ ]:
%%bash
curl --request GET \
  --url https://api.surveymonkey.com/v3/webhooks \
  --header 'Accept: application/json' \
  --header 'Authorization: Bearer {access-token}'

In [ ]:
## Get Triggered by Webhook (TO-DO)
def webhook_trigger() -> dict: 
    """Placeholder for SurveyMonkey webhook endpoint/trigger. 
    Right now returns example 'response_completed' event, taken from API documentation."""

    response_event = {
    "name": "My Webhook",
    "filter_type": "collector",
    "filter_id": "123456789",
    "event_type": "response_completed",
    "event_id": "123456789",
    "object_type": "response",
    "object_id": "123456",
    "event_datetime": "2016-01-01T21:56:31.182613+00:00",
    "resources": {
        "respondent_id": "114409718452", # Replaced this with Kamran's response 
        "recipient_id": "123456789",
        "collector_id": "123456789",
        "survey_id": "123456789",
        "user_id": "123456789"
        }
    }
    re

In [17]:
## GET/Load question-answer keys for SurveyMonkey Survey and CareerOneStop Skills Matcher
def get_details(source=None, fetch=False) -> dict:
    """
    Load list of questions/answers from either the Survey Monkey API `/details` endpoint or from CareerOneStop. 

    Args: 

    source (str):   Must be one of 'sm' (for Skills Monkey Survey) or 'cos' (for CareerOneStop)

    fetch (bool):   Whether to GET new question/answer key from source or to just use locally saved copy (default=False). 

          If fetch == True:
            If the GET request fails, we use our cached copy.
            If the question/answer details have changed, we update our cached copy. Any such changes may break create_map() and this app as a whole.

        Note 500 requests/month limit to SM -- if going to use fetch option, may want to only do so periodically.

    """

    # Set SM vs. COS variables
    if source == "sm": 
        url = f"{SM_DATA['base_url']}/details"
        headers = SM_DATA['headers']
        cached_fp = SM_DATA['survey-details-fp']
    elif source == "cos":
        url = COS_DATA['url']
        headers = COS_DATA['headers']
        cached_fp = COS_DATA['survey-details-fp']
    else:
        raise Exception("`source` must be one of `sm` (SurveyMonkey) or `cos` (CareerOneStop)")
    
    # Load cached details 
    with open(cached_fp, "r") as file: 
        cached_details = json.load(file)

    ## Request (if fetch == True)
    # Attempt request for survey details
    if fetch: 
        # If wrong response code, or request caused error, solely use cached details file
        request_fail = False
        try: 
            response = get_request(url=url, headers=headers)
            if response.status_code != 200:
                log_azure(f"WARNING: GET {source.upper()} survey details -- Response Code: {response.status_code} -- Proceeding with cached file: {cached_fp}")
                request_fail = True 
        except Exception as e: 
            log_azure(f"ERROR: GET {source.upper()} survey details -- Error: {str(e)} -- Proceeding with cached file: {cached_fp}")
            request_fail = True
        if request_fail: 
            return cached_details
        # If request successful, check if newer than cached copy
        else:   
            fetched_details = response.json()
            if fetched_details['date_modified'] != cached_details['date_modified']: ## TO-DO: check to make sure date_modified types match before comparing
                log_azure(f"WARNING: GET {source.upper()} survey details -- Response modified ({fetched_details['date_modified']}) since last use ({cached_details['date_modified']} -- Updating cached file: {cached_fp}.")           
                with open(cached_fp, "w") as file: 
                    json.dump(fetched_details, file)
            return fetched_details
        
    ## Just load (if fetch == False)
    else: 
        return cached_details

In [95]:
## Create translation map from Survey Monkey key to COS key 
def create_map() -> dict: 
    
    ## Get question/answer keys 
    sm_key = get_details("sm")
    cos_key = get_details("cos")

    ## Prepare translation map
    combined_map = {
        'non-skills-matcher':[], # not to send to COS (background questions)
        'skills-matcher':[], # to send to COS skills matcher 
    }

    ## Adding relevant SM information to map
    for p in sm_key['pages']:
        page_title = p['title'].lower() 
        for q in p['questions']:
            question_id = q['id']
            question_type = 'skills-matcher' if "skills matcher" in page_title else "non-skills-matcher" # fails for demo -- limited to one page (see line 35)
            question_num = q['position']
            question_text = [h['heading'] for h in q['headings']] # for human readability/checking
            if 'answers' in q.keys(): 
                answer_ids = [d['id']
                    for d in q['answers']['choices']] # will zip these with the COS answer ids in next step
            else:
                answer_ids = None

            combined_map[question_type].append({
                'question_id':{'sm':question_id},
                'question_number':{'sm':question_num},
                'question_text':{'sm':question_text},
                'answer_ids':answer_ids
            })

    # Moving skills matcher/non skills matcher questions for demo, since stuck to one page (line 19) (TO-DO: remove when using official survey)
    combined_map['skills-matcher'] = combined_map['non-skills-matcher'][1:]
    combined_map['non-skills-matcher'] = [combined_map['non-skills-matcher'][0]]

    ## Adding relevant COS information 
    # Check that the number of questions to send to the COS API matches the expected amount -- this would trip on test run with demo survey
    # if len(combined_map['skills-matcher']) != len(cos_key['Skills']):
    #     log_azure(f"ERROR: No. of skills-matcher questions retrieved from SM {len(combined_map['skills-matcher'])} doesn't match number in COS {len(cos_key['Skills'])}")
    #     raise Exception

    for n in range(len(combined_map['skills-matcher'])): # len(combined_map['skills-matcher']) == len(cos_key['Skills'])
        cos_q = cos_key['Skills'][n]
        cos_answer_ids = [cos_q["DataPoint20"],
                        cos_q["DataPoint35"], 
                        cos_q["DataPoint50"], 
                        cos_q["DataPoint65"], 
                        cos_q["DataPoint80"]]

        combined_map['skills-matcher'][n]['question_id']['cos'] = cos_q['ElementId']
        combined_map['skills-matcher'][n]['question_number']['cos'] = n + 1
        combined_map['skills-matcher'][n]['question_text']['cos'] = cos_q['Question']
        combined_map['skills-matcher'][n]['answer_ids'] = dict(zip(combined_map['skills-matcher'][n]['answer_ids'], cos_answer_ids))

    ## Casting question lists to dictionary for easier lookup in translation -- keeping these as lists made the previous insertion step easier
    combined_map['skills-matcher'] = {q['question_id']['sm']:q for q in combined_map['skills-matcher']}
    combined_map['non-skills-matcher'] = {q['question_id']['sm']:q for q in combined_map['non-skills-matcher']}

    return combined_map

combined_map = create_map()

In [ ]:
## Translate response from SM to COS format

In [ ]:
## GET COS Job recommendations 

In [1]:
import json
with open("survey-details.json", "r") as file: 
    cached_details = json.load(file)

cached_details

{'title': 'Career Onestop Port',
 'nickname': '',
 'language': 'en',
 'folder_id': '0',
 'category': '',
 'question_count': 10,
 'page_count': 1,
 'response_count': 1,
 'date_created': '2023-09-08T17:25:00',
 'date_modified': '2023-09-13T15:03:00',
 'id': '409346397',
 'buttons_text': {'next_button': 'Next',
  'prev_button': 'Prev',
  'done_button': 'Done',
  'exit_button': ''},
 'is_owner': True,
 'footer': True,
 'theme_id': '10292568',
 'custom_variables': {},
 'href': 'https://api.surveymonkey.com/v3/surveys/409346397',
 'analyze_url': 'https://www.surveymonkey.com/analyze/JrhdBA97A18icLNdv_2B26M4cs4Lx9WaWryCJ3TK_2F_2FzUk_3D',
 'edit_url': 'https://www.surveymonkey.com/create/?sm=JrhdBA97A18icLNdv_2B26M4cs4Lx9WaWryCJ3TK_2F_2FzUk_3D',
 'collect_url': 'https://www.surveymonkey.com/collect/list?sm=JrhdBA97A18icLNdv_2B26M4cs4Lx9WaWryCJ3TK_2F_2FzUk_3D',
 'summary_url': 'https://www.surveymonkey.com/summary/JrhdBA97A18icLNdv_2B26M4cs4Lx9WaWryCJ3TK_2F_2FzUk_3D',
 'preview': 'https://www.s

In [ ]:
## Make GET request to Skill Monkey for survey responses 
# Make request 
# Filter for new responses (we only have one response)

In [ ]:
## Ta